In [1]:

from arweave.arweave_lib import Wallet, Transaction, API_URL
from arweave.transaction_uploader import get_uploader
import json
import os
from tqdm import tqdm
import cv2

# Load your arweave wallet
your_ar_wallet = Wallet('arweave-key.json')
your_ar_wallet.address, your_ar_wallet.balance

('sGkCs5By-CKEYV83z04qNc66Q_X90YEuL6J9fhfICi0', 0.544663347664)

In [3]:
Classes = ["Ruby", "Diamond", "Sapphire", "Emerald", "Serendibite", "Benitoite"]

Report = {}
for ingl_class in tqdm(Classes):

    arr = os.listdir(f'./Gems/{ingl_class}')
    t_dict = {}

    for vid in arr:

        with open(f'./Gems/{ingl_class}/{vid}', 'rb') as file_handler:
            tx = Transaction(your_ar_wallet, file_handler=file_handler, file_path=f'./Gems/{ingl_class}/{vid}')
            tx.add_tag('Content-Type', 'video/mp4')
            tx.sign()

            uploader = get_uploader(tx, file_handler)

            while not uploader.is_complete:
                uploader.upload_chunk()

                # logger.info("{}% complete, {}/{}".format(
                #     uploader.pct_complete, uploader.uploaded_chunks, uploader.total_chunks
                # ))

        vid_url = API_URL+"/"+tx.id

        imgs = cv2.VideoCapture(f'./Gems/{ingl_class}/{vid}')
        imgs.set(1,155)
        ret, frame = imgs.read()
        cv2.imwrite(f'./Gem_images/{ingl_class}_{vid.split(".")[0]}.jpg', frame)

        with open(f'./Gem_images/{ingl_class}_{vid.split(".")[0]}.jpg', 'rb') as f:
            img_in_bytes = f.read()

        transaction = Transaction(your_ar_wallet, data=img_in_bytes)
        transaction.add_tag('Content-Type', 'image/png')
        transaction.sign()
        transaction.send()

        image_url = API_URL+"/"+transaction.id

        metadata = {
        "name": "Ingl-Gem",
        "symbol": "I-Gem",
        "description": "Democratizing The Process of Creating Solana Validators",
        "seller_fee_basis_points": 300,
        "external_url": "https://www.ingl.io",
        "attributes": [
            {
                "trait_type": "Generation",
                "value": "1",
            },
            {
                "trait_type": "Class",
                "value": ingl_class,
            },
            {
                "trait_type": "Rarity",
                "value": vid.split(".")[0],
            },
        ],
        "collection": {
            "name": "Ingl-Gem",
            "family": "Ingl Collections",
        },
        "properties": {
            "files": [
                {
                    "uri": vid_url,
                    "type": "video/mp4",
                },
            ],
            "category": "video",
            "maxSupply": None,
            "creators":[],
        },
        "image": image_url,
        }


            # Upload metadata to Arweave
        meta_transaction = Transaction(your_ar_wallet, data=json.dumps(metadata))
        meta_transaction.add_tag('Content-Type', 'text/html')
        meta_transaction.sign()
        meta_transaction.send()

        metadata_url = API_URL+"/"+meta_transaction.id
        t_dict[vid.split(".")[0]] = metadata_url
    Report[ingl_class] = t_dict
with open("uploaded_images.json", "w") as outfile:
    json.dump(Report, outfile)
Report

100%|██████████| 1/1 [01:13<00:00, 73.47s/it]


{'Emerald': {'Mythic': 'https://arweave.net/YrEYF6RBkZCSMvBXJkw63g4WienAGGOlHqOl3QdTr1E'}}